<a href="https://colab.research.google.com/github/Mehak-shahani/my-projects-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer
import torch
from sentence_transformers import util


In [ ]:
# Load a faster model
faq_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Smaller model
faq_model = faq_model.to('cuda' if torch.cuda.is_available() else 'cpu')  # Use GPU if available

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
faq_data = [
    {"question": "What is your return policy?", "answer": "Our return policy allows returns within 30 days of purchase."},
    {"question": "How do I track my order?", "answer": "You can track your order by entering your tracking number on our website."},
    {"question": "What payment methods do you accept?", "answer": "We accept credit cards, PayPal, and bank transfers."},
]

In [ ]:
# Create embeddings for FAQ questions
faq_questions = [faq['question'] for faq in faq_data]
faq_embeddings = faq_model.encode(faq_questions, convert_to_tensor=True)

In [10]:
# Ask the user for input
user_input = input("Please enter your question: ")

# Compute the embedding for the user input
user_embedding = faq_model.encode(user_input, convert_to_tensor=True)

# Compute the cosine similarities between the user input and the FAQ questions
similarities = util.pytorch_cos_sim(user_embedding, faq_embeddings)[0]

# Get the index of the FAQ with the highest similarity score
best_match_idx = similarities.argmax()

# Get the best score for the matched FAQ
best_score = similarities[best_match_idx].item()
if best_score > 0.7:
    print(f"Chatbot: {faq_data[best_match_idx]['answer']}")
else:
    print("Chatbot: I'm not sure about that. Let me pass your query to our human support team.")

Please enter your question: What payment methods do you accept
Chatbot: We accept credit cards, PayPal, and bank transfers.
